In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import tensorflow

- List available devices

In [ ]:
tensorflow.config.experimental.list_physical_devices()

- Check if build with cuda (cuda compatible)

In [ ]:
tensorflow.test.is_built_with_cuda()

# 1. Import –  necessary modules and the dataset.


In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [7]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()



# 2. Perform  Eda – check data and labels shape:


In [ ]:

# checking images shape
X_train.shape, X_test.shape


In [ ]:

# display single image shape
X_train[0].shape

 


In [ ]:

# checking labels
y_train[:5]

# 3. Apply Preprocessing: 
Scaling images(NumPy array) by 255 and One-Hot Encoding labels to represent all categories as 0, except  1 for the actual label in ‘float32.’


In [11]:

# scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255

# one hot encoding labels
# y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
# y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10)
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10)


# 4. Model Building: 
A fn to build a neural network with architecture as below with compiling included :


In [12]:

def get_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')    
    ])
    model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

# 5. Training: 
Train for ten epochs which verbose = 0, meaning no logs.



## CPU
 here -n1 -r1 will ensure the process will run for only one pass, not specifying will perform runs for few no of times and then calculate the average. 
 
 Also (CPU:0) refers to the first CPU(I have only one).


In [ ]:
%%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_cpu = get_model()
    model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 10)


## GPU

In [ ]:

%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

# TEST Two – Training Clothes Classifier

In [ ]:
# loading dataset
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# checking shape

print(train_images.shape)

print(train_labels[0])

In [ ]:
# checking images
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
plt.imshow(train_images[0])
class_names[train_labels[0]]

In [17]:
# scaling
train_images_scaled = train_images / 255.0
test_images_scaled = test_images / 255.0

# Finally, let’s change our model to define how many hidden layers we need for our work and set remote units to be 500. 
# This will allow for a tuning model architecture in case of model overfits.

def get_model(hidden_layers=1):
    # Flatten layer for input
    layers = [keras.layers.Flatten(input_shape=(28, 28))]
    # hideen layers
    for i in range(hidden_layers):
        layers.append(keras.layers.Dense(500, activation='relu'),)
    # output layer    
    layers.append(keras.layers.Dense(10, activation='sigmoid'))
    model = keras.Sequential(layers)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

If this seems unfamiliar, let me break it down:

In the above code, we store layers as a list and then append those hidden layers as provided in the hidden_layers. Finally, we compile our model with adam as optimizer and sparce_categorical_crossentropy as loss fn. Metric to monitor is again accuracy.

Finally, let’s train our model with 5 hidden layers for 5 epochs:

## CPU

In [ ]:
%%timeit -n1 -r1
with tf.device('/CPU:0'):
    cpu_model = get_model(hidden_layers=5)
    cpu_model.fit(train_images_scaled, train_labels, epochs=5)

# GPU

In [ ]:
%%timeit -n1 -r1
with tf.device('/GPU:0'):
    gpu_model = get_model(hidden_layers=5)
    gpu_model.fit(train_images_scaled, train_labels, epochs=5)